# install libraries

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

# import all required libaries

In [3]:
import torch 
import os
from datasets import load_dataset
from transformers import(AutoModelForCausalLM,
                        AutoTokenizer,
                        BitsAndBytesConfig,
                        HfArgumentParser,
                        TrainingArguments,
                        pipeline,
                        logging,
                )
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# in LLma2, following promt template is used for chat model

System prompt(optional) to guide the model,
user promt(required),
answer the model(required)

In [4]:
# model that you want to train from the hugging face hub
model_name="NousResearch/Llama-2-7b-chat-hf"

# instruct dataset to use 
dataset_name="mlabonne/guanaco-llama2-1k"

# fined tuned model name
new_model="Llama-2-7b-chat-finetune"

# QloRA Parameters

# LoRA attention dimension
lora_r=64

# Alpha parameter for LoRA scaling 
lora_alpha=16

# Dropout probability for LoRA layer

lora_dropout=0.1

# bitsandbytes parameters 
# activate 4-bit precision base model loading

use_4bit=True

# compute dtype for 4-bit models
bnb_4bit_compute_dtype="float16"

# quantization type (fp4 0r nf4)
bnb_4bit_quant_type="nf4"

# activate nested quantization for 4-bit base models (double quantization)
use_nested_quant=False


## training arguments parameters

# output directory where the model predictions and checkpoints will be stored
output_dir="./results"

# Number of training epochs
num_train_epochs=1

# Enable fp16/bf16 training (set bf16 to True  with an A100)
fp16=False
bf16=False

# Batch size per GPU for training
per_device_train_batch_size=4

# batch size per GPU for evaluation
per_device_eval_batch_size=4

# Number of update steps to accumate the gradient for 
gradient_accumation_steps=1

#Enable gradient checkpointing
gradient_checkpointing=True

# Maximum gradient normal (gradient Clipping)
max_grad_norm=0.3

# Initial Learning rate (AdamW Optimizer)
learning_rate=2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay=0.001

# Optimizer to use
optim="paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type="cosine"

# Number of training steps (overrides num_train_epochs)
max_step=-1

# Ratio of steps for linear warmup (from 0 to learning rate)
warmup_ratio=0.03

# Group sequences into batches with same lenght
# Saves memory and speeds up training considerably
group_by_length=True

# save checkpoints every X updates steps 
save_steps=0

# Log every X updates steps 
logging_steps=25

# SFT parameters 

# Maximum sequence length to use
max_seq_length=None

# pack multiple short examples in the same inputs sequence to increase efficiency 
packing=False

# Load the entire model on the GPU 0
device_map={"":0}

# Load Everything and start the fine_tuning process

In [5]:
# Load Datasets (you can process it here)
dataset=load_dataset(dataset_name,split="train")

# Load Tokenizer and model with QLoRA configuration 
compute_dtype=getattr(torch,bnb_4bit_compute_dtype)

bnb_config=BitsAndBytesConfig(
     load_in_4bit=use_4bit,
     bnb_4bit_quant_type=bnb_4bit_quant_type,
     bnb_4bit_compute_dtype=compute_dtype,
     bnb_4bit_use_double_quant=use_nested_quant,
)
# Check GPU compatibility with bfloats16
if compute_dtype==torch.float16 and use_4bit:
    major,_=torch.cuda.get_device_capability()
    if major>=8:
        print("="*80)
        print("Your GPU Support bfloat16: accelerate training with bf16=True")
        print("="*80)
        
# Load base model
model=AutoModelForCasualLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache=False
model.config.pretraining_tp=1

## Load LLaMa tokenizer
tokenizer=AutoTokenizer.from_pretarined(model_name,trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right" # Fix weired overflow issue with fp16 training

# Load loRA configuration
peft_config=LoraConfig(
     lora_alpha=lora_alpha,
     loar_dropout=lora_dropout,
     r=lora_r,
     bias="none",
     task_type="CASUAL_LM"
)

#set training parameters
training_arguments=Training_Arguments(
      output_dir=output_dir,
      num_train_epochs=num_train_epochs,
      per_device_train_batch_size=per_device_train_batch_size,
      gradient_accumulation_steps=gradient_accumulation_steps,
      optim=optim,
      save_steps=save_steps,
      logging_steps=logging_steps,
      learning_rate=learning_rate,
      weight_decay=weight_decay,
      fp16=fp16,
      bf16=bf16,
      max_grad_norm=max_grad_norm,
      max_steps=max_steps,
      warmup_ratio=warmup_ratio,
      group_by_length=group_by_length,
      lr_scheduler_type=lr_scheduler_type,
      report_to="tensorboard"
)
# set supervised fine tuning parameters
trainer=SFTTrainer(
       model=model,
       train_dataset=dataset,
       peft_config=peft_config,
       dataset_text_field="text",
       max_seq_length=max_seq_length,
       tokenizer=tokenizer,
       args=training_arguments,
       packing=packing,
)

# Train model
trainer.train()

Found cached dataset parquet (C:/Users/Asus/.cache/huggingface/datasets/mlabonne___parquet/mlabonne--guanaco-llama2-1k-f1f1134768f90029/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx